# Data Cleaning for Tableau and Model Input

In [14]:
#import necessary libraries and set required options for display
import pandas as pd
import pandas_profiling
import string as str
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 700)

In [15]:
#Import the data
df_app_info = pd.read_csv("df_app_info_work.csv",encoding = 'latin1')
#Remove non informative columns
df_app_info = df_app_info.drop(["Unnamed: 0","app_id","current_version","description","description_html",
                                "developer_id","recent_changes","screenshots","developer_address",
                               "developer_email","developer_url","histogram","video",
                                "icon","iap_range",'interactive_elements',"url"],axis = 1)

#Remove Duplicates
df_app_info.drop_duplicates(inplace = True,subset = "title")

In [16]:
df_app_info.columns = ['CATEGORY','CONTENT_RATING','DEVELOPER','EDITORS_CHOICE',
                       'FREE','IAP','INSTALLS','PRICE','REQUIRED_ANDROID_VERSION','NUMBER_REVIEWS','RATING_SCORE',
                       'SIZE','TITLE','LAST_UPDATED']

In [17]:
df_app_info.head()

,CATEGORY,CONTENT_RATING,DEVELOPER,EDITORS_CHOICE,FREE,IAP,INSTALLS,PRICE,REQUIRED_ANDROID_VERSION,NUMBER_REVIEWS,RATING_SCORE,SIZE,TITLE,LAST_UPDATED
0,['COMMUNICATION'],['Everyone'],Facebook,True,True,True,"1,000,000,000+",0,Varies with device,67225837,4.1,Varies with device,Messenger  Text and Video Chat for Free,26-Jun-19
1,['MUSIC_AND_AUDIO'],['Teen'],Spotify Ltd.,False,True,False,"500,000,000+",0,Varies with device,15465761,4.6,Varies with device,"Spotify: Discover music, podcasts, and playlists",25-Jun-19
2,['MUSIC_AND_AUDIO'],['Teen'],Pandora,False,True,True,"100,000,000+",0,Varies with device,3073185,4.4,Varies with device,"Pandora - Streaming Music, Radio & Podcasts",26-Jun-19
3,['MAPS_AND_NAVIGATION'],['Everyone'],"Uber Technologies, Inc.",False,True,False,"100,000,000+",0,Varies with device,6290761,4.2,Varies with device,Uber,24-Jun-19
4,['WEATHER'],['Everyone'],The Weather Channel,True,True,True,"100,000,000+",0,Varies with device,1885387,4.4,Varies with device,Weather Maps & Storm Radar - The Weather Channel,25-Jun-19


In [18]:
#Check for missing values and remove/ impute
#df_app_info.isnull().sum()

In [19]:
#There are ~900 apps with missing rating_score. Since this is our explaned variable, we'll exclude these apps 
df_app_info = df_app_info[pd.notnull(df_app_info['RATING_SCORE'])]

#There are 12 apps with REQUIRED_ANDROID_VERSION missing. We'll exclude these apps
df_app_info = df_app_info[pd.notnull(df_app_info['REQUIRED_ANDROID_VERSION'])]

#There is 1 app with number of installs missing. We'll exclude that app
df_app_info = df_app_info[pd.notnull(df_app_info['DEVELOPER'])]

#There is 1 app with developer information missing. We'll exclude that app
df_app_info = df_app_info[pd.notnull(df_app_info['INSTALLS'])]

In [20]:
#Check again if all is good now!
#df_app_info.isnull().sum()

In [21]:
# Clean up "Category"
df_app_info['CATEGORY'] = df_app_info['CATEGORY'].map(lambda x: x.replace("['",""))
df_app_info['CATEGORY'] = df_app_info['CATEGORY'].map(lambda x: x.replace("']",""))
df_app_info['CATEGORY'] = df_app_info['CATEGORY'].map(lambda x: x.replace("'",""))
df_app_info['CATEGORY'] = df_app_info['CATEGORY'].map(lambda x: x.split(",")[0])

# Clean up "Content_Rating"
df_app_info['CONTENT_RATING'] = df_app_info['CONTENT_RATING'].map(lambda x: x.replace("['",""))
df_app_info['CONTENT_RATING'] = df_app_info['CONTENT_RATING'].map(lambda x: x.replace("']",""))
df_app_info['CONTENT_RATING'] = df_app_info['CONTENT_RATING'].map(lambda x: x.replace("'",""))
df_app_info['CONTENT_RATING'] = df_app_info['CONTENT_RATING'].map(lambda x: x.split(",")[0])

# Cleanup up "Price"
df_app_info['PRICE'] = df_app_info['PRICE'].apply(lambda x : x.strip('$')).astype(float)

# Cleanup up "Installs"
df_app_info['INSTALLS'] = df_app_info['INSTALLS'].apply(lambda x : x.strip('+').replace(',', '')).astype(float)


#Cleanup up "REQUIRED_ANDROID_VERSION"
df_app_info.loc[df_app_info['REQUIRED_ANDROID_VERSION'] == 'Varies with device','REQUIRED_ANDROID_VERSION'] = '0' 
df_app_info['REQUIRED_ANDROID_VERSION'] = df_app_info['REQUIRED_ANDROID_VERSION'].str[0:3].astype(float)

In [22]:
#Cleanup up "Size"

#First let's convert KBs to MBs
kbs = df_app_info['SIZE'].loc[df_app_info['SIZE'].str.contains('k')].index.tolist()
Kb_to_Mb = pd.DataFrame(df_app_info.loc[kbs,'SIZE'].apply(lambda x: x.strip('k')).astype(float)
                          .apply(lambda x: x / 1024).apply(lambda x: round(x, 3)))

#Now Strip M 
Kb_to_Mb['SIZE'] = Kb_to_Mb['SIZE'].astype('str')
df_app_info.loc[kbs,'SIZE'] = Kb_to_Mb
df_app_info['SIZE'] = df_app_info['SIZE'].apply(lambda x: x.strip('M'))

#If size varies with device, mark it as 0 and conver it to float
df_app_info.loc[df_app_info['SIZE'] == 'Varies with device','SIZE'] = 0 
df_app_info['SIZE'] = df_app_info['SIZE'].astype(float)


In [23]:
#df_app_info.head()

In [24]:
#Univariate Analysis Report
pandas_profiling.ProfileReport(df_app_info)

Number of variables,15
Number of observations,40218
Total Missing (%),0.0%
Total size in memory,3.8 MiB
Average record size in memory,99.0 B
Numeric,7
Categorical,4
Boolean,3
Date,0
Text (Unique),1
Rejected,0


In [25]:
# Number of installs is highly skewed and contains outliers with 95th percentile = 10000000 and Maximum = 5000000000.
# We will exclude apps where number of installs < 5th percentile and number of installs > 95th percentile

app_data_informative = df_app_info[(df_app_info['INSTALLS'] > 100) & (df_app_info['INSTALLS'] <= 10000000)]
app_data_informative.shape

(36557, 14)

In [26]:
#Push the dataframe into a csv for Tableau and Model input
app_data_informative.to_csv("app_data_informative_excluding_outliers.csv")